In [5]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

# Selenium Document


- Selenium document
https://selenium-python.readthedocs.io/locating-elements.html

# Selenium 사용 간단한 예

In [5]:
url = "http://www.naver.com"                # url 지정
driver = webdriver.Chrome('chromedriver')   # chrom browser 열기, chromedriver.exe 파일이 있어야 함
driver.get(url)                             # 지정된 url 열기

In [14]:
element_comment = driver.find_element_by_class_name("btn_more")   # 작동시키고 싶은 객체 찾기
element_comment.click()                                               # 원하는 종류의 작동 (link나 button형태로 되어 있으면 click 가능)

In [15]:
element_comment = driver.find_element_by_name("movie")   # 작동시키고 싶은 객체 찾기
element_comment.click()                                               # 원하는 종류의 작동 (link나 button형태로 되어 있으면 click 가능)

In [20]:
url = "http://www.naver.com"                # url 지정
driver.get(url) 
element_id = driver.find_element_by_id('query')   # 작동시키고 싶은 객체 찾기
element_id.send_keys("python selenium")  
element_search = driver.find_element_by_class_name('ico_search_submit')   # 작동시키고 싶은 객체 찾기
element_search.submit()                            # 자료 입력 효과

In [21]:
driver.close()

# Selenium 이용한 뉴스 기사 개요와 댓글 scraping

In [11]:
driver = webdriver.Chrome('chromedriver')   # Selenium

In [12]:
Article_df = pd.DataFrame(columns=['news_site', 'post_id', 'url', 'date', 'time', 'title', 'writer',  
                                   'cmt_count'])
Comment_df = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 'user_name',
                                   'user_id','yes', 'no', 'cmt_text'])

Article_df_temp = pd.DataFrame(columns=['news_site', 'post_id', 'url', 'date', 'time', 'title', 'writer',  
                                   'cmt_count'])
Comment_df_temp = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 'user_name',
                                   'user_id','yes', 'no', 'cmt_text'])

filename = "18_link_chosun_sample.xlsx"
df_url_list = pd.read_excel(filename)

i = 0
while i < len(df_url_list):
    post_id = df_url_list.iloc[i].post_id
    url = df_url_list.iloc[i].link
    print("processing: ", url)
    i += 1
    if 'chosun' in url: 
        try:
            driver.get(url)   
        except:
            print("----처리 불가능한 site\n")
            continue

        news_site='chosun'
        Article_df_temp, Comment_df_temp  = chosun(url, post_id) 
        Article_df = Article_df.append(Article_df_temp)
        Comment_df = Comment_df.append(Comment_df_temp)
        print("----Done\n")

writer = pd.ExcelWriter('article.xlsx', engine='xlsxwriter')
Article_df.to_excel(writer)
writer.save() 

writer = pd.ExcelWriter('comment.xlsx', engine='xlsxwriter')
Comment_df.to_excel(writer)
writer.save() 

processing:  http://news.chosun.com/site/data/html_dir/2017/09/10/2017091002294.html?fbclid=IwAR3WeBo4jYHtFZrTnPckLDDPJccC-DzS3rLl8Ryzfo8H6WO2l0sYwzRFufU
----Done

processing:  https://biz.chosun.com/site/data/html_dir/2019/11/17/2019111700344.html
----Done

processing:  http://news.chosun.com/site/data/html_dir/2019/10/25/2019102500045.html
----Done



In [8]:
def chosun(url, post_id):
    #make DataFrame
    Article_df = pd.DataFrame(columns=['news_site', 'post_id','url', 'date', 'time', 'title', 'writer', 'cmt_count'])
    Comment_df = pd.DataFrame(columns=['post_id', 'cmt_id', 'cmt_date', 'cmt_time', 
                                       'user_name','user_id','yes', 'no', 'cmt_text'])

    #parsed html
    html = driver.page_source
    parsed = BeautifulSoup(html, "html.parser")

    ## 기사 본문
    #기사 title
    try: 
        title = parsed.find('div', {'class': "layout--bg"}).text
    except:
        title = "null"

    #기사 입력시간
    try:
        date_time = parsed.find('div', {'class':"news_date"}).text.split('|')[0].strip().split(" ")
        date = date_time[1]
        time = date_time[2]
    except:
        date = "null"
        time = "null"
        
    #기자, 작성자
    try:
        writer_info = parsed.find('li', {'id':"j1"}).text.strip().split(" ")
        writer = writer_info[0]
    except:
        writer = "null"

    # 총 댓글 수 파악
    try:
        cmt_count_written = parsed.find('em', {'class': 'cmt_count'}).text
    except:
        cmt_count_written = 0

    # 기사 DF 추가
    Article_df = Article_df.append({'news_site' : 'chosun', 'post_id': post_id, 'url': url, 
                                    'date': date, 'time': time, 'title': title, 'writer': writer, 
                                    'cmt_count':cmt_count_written}, ignore_index = True)

    #move comment page (chosun)
    try:
        element_comment = driver.find_element_by_class_name("cmt_count")   #Selenium
        element_comment.click()                                            #Selenium
    except:
        pass

    ## chosun comment scraping
    # 첫번째 page
    page_n = 1

    #parsed html
    html = driver.page_source
    parsed = BeautifulSoup(html, "html.parser")

    try:
        comments = parsed.findAll('div', {'class': 'cmt_box'})
        cmt_count=len(comments) #일반 댓글 수
       
    except:
        cmt_count = 0

    cmt_no = 0
    for cmt in comments:
        cmt_no += 1
        cmt_id = ('%s_C%s' %(post_id, cmt_no))
        
        yes = 0
        no = 0
        
        try:
            # 여기 작성하시오
            '''
            cmt_date = 
            cmt_time = 
            user_name = 
            user_id = 
            text = 
            yes = 
            no = 
            '''
            Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id,  
                                            'cmt_date': cmt_date, 'cmt_time': cmt_time, 
                                            'user_name': user_name, 'user_id': user_id, 'yes': yes, 'no': no, 
                                            'cmt_text': text}, ignore_index = True)
        except:
            Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id, 
                                            'cmt_date': "null", 'cmt_time': "null", 
                                            'user_name': "null",'user_id': "null",'yes': "null", 'no': "null", 
                                            'cmt_text': "삭제된글"}, ignore_index = True)
        
    # 넘어가야 할 페이지가 더 있는지 확인    
    try:
        pages = parsed.find('ul', {'class': 'paginate_num'})
        num_a = pages.findAll('a')
        n = len(num_a) - 1     # 이전, 현재, 다음도 a로 묶여 있으므로 이들 제외하고 넘어가야 하는 페이지 수
        page_links = [pages.findAll('a')[i]['href'] for i in range(2, n)]  # 세번째 a, 즉 [2]부터 link가 있게 됨
        #print(page_links)
    except:
        page_links = []

    # 있다면 이후 작업
    for page in page_links:
        page_n += 1

        driver.get(page)
        html = driver.page_source

        parsed = BeautifulSoup(html, "html.parser")  
        comments = parsed.findAll('div', {'class': 'cmt_box'})
        cmt_count += len(comments)

        #print(comments)

        #print('%d 페이지 댓글 수: %d' %(page_n, len(comments)))

        for cmt in comments:
            cmt_no += 1
            cmt_id = ('%s_C%s' %(post_id, cmt_no))
            yes = 0
            no = 0
            try:
                 # 여기 작성하시오
                '''
                cmt_date = 
                cmt_time = 
                user_name = 
                user_id = 
                text = 
                yes = 
                no = 
                '''
                Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id, 
                                            'cmt_date': cmt_date, 'cmt_time': cmt_time, 
                                            'user_name': user_name, 'user_id': user_id, 'yes': yes, 'no': no, 
                                            'cmt_text': text}, ignore_index = True)
            except:
                Comment_df = Comment_df.append({'post_id': post_id, 'cmt_id': cmt_id,  
                                            'cmt_date': "null", 'cmt_time': "null", 
                                            'user_name': "null",'user_id': "null",'yes': "null", 'no': "null", 
                                            'cmt_text': "삭제된글"}, ignore_index = True)
    return Article_df, Comment_df

In [13]:
Article_df

,news_site,post_id,url,date,time,title,writer,cmt_count
0,chosun,chosun01,http://news.chosun.com/site/data/html_dir/2017...,null,null,2021년 6월 15일(화)中文|English|日本語뉴스홈백신을 맞읍시다오피니언프리...,null,0
0,chosun,chosun02,https://biz.chosun.com/site/data/html_dir/2019...,null,null,로그인2021년 6월 15일(화)中文|English|日本語증권부동산ITCar금융...,null,0
0,chosun,chosun03,http://news.chosun.com/site/data/html_dir/2019...,null,null,2021년 6월 15일(화)中文|English|日本語뉴스홈백신을 맞읍시다오피니언프리...,null,0


In [28]:
Comment_df

,post_id,cmt_id,cmt_date,cmt_time,user_name,user_id,yes,no,cmt_text
